In [142]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [143]:
import numpy as np
import os
import sys
import random
import os
import string
import json

sys.path.append(os.path.abspath(os.path.join(os.path.pardir, 'src')))

from concepts import DynamicConcepts, linear_regression
from policy import ConvNet

model_name = "net"
session_name = "endeavor"
board_size = 5
board_name = f'{board_size}x{board_size}'
simulations = 3000
concept_type_single = False # If True, mcts will only consider states for the root player.

agents_to_sample = [0, 100, 200, 300, 400, 500, 600, 700, 800, 900, 1000]

full_model_path = f"../models/saved_sessions/board_size_{board_size}/{session_name}/"

mcts_model_path = f"../models/best_models/board_size_{board_size}/net_1000.keras"

init_state = DynamicConcepts.opening_play(board_size)

dynamic_concept = DynamicConcepts(init_state, simulations, board_size, concept_type_single, mcts_model_path)

CONCEPT_NAME = DynamicConcepts.opening_play.__name__

INFO:tensorflow:Assets written to: /var/folders/k4/t2xnyks517g9v60tdd71p4_00000gn/T/tmp2ovscbyr/assets


INFO:tensorflow:Assets written to: /var/folders/k4/t2xnyks517g9v60tdd71p4_00000gn/T/tmp2ovscbyr/assets
2024-02-07 16:28:15.772348: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:364] Ignored output_format.
2024-02-07 16:28:15.772366: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:367] Ignored drop_control_dependency.
2024-02-07 16:28:15.772560: I tensorflow/cc/saved_model/reader.cc:45] Reading SavedModel from: /var/folders/k4/t2xnyks517g9v60tdd71p4_00000gn/T/tmp2ovscbyr
2024-02-07 16:28:15.775255: I tensorflow/cc/saved_model/reader.cc:89] Reading meta graph with tags { serve }
2024-02-07 16:28:15.775267: I tensorflow/cc/saved_model/reader.cc:130] Reading SavedModel debug info (if present) from: /var/folders/k4/t2xnyks517g9v60tdd71p4_00000gn/T/tmp2ovscbyr
2024-02-07 16:28:15.784518: I tensorflow/cc/saved_model/loader.cc:231] Restoring SavedModel bundle.
2024-02-07 16:28:15.830935: I tensorflow/cc/saved_model/loader.cc:215] Running initialization

In [144]:
def load_model(full_name, model_name, epoch):
    model_path = full_name + model_name + "_" + str(epoch) + ".keras"
    model = ConvNet(board_size, model_path)
    return model

agents = [load_model(full_model_path, model_name, epoch) for epoch in agents_to_sample]

In [145]:
positive_cases = []
negative_cases = []

pos, neg = dynamic_concept.generate_cases()

positive_cases.extend(pos)
negative_cases.extend(neg)

# Create numpy arrays 
positive_cases = np.array(positive_cases)
negative_cases = np.array(negative_cases)

In [146]:
# Print shapes
print("Positive cases: ", positive_cases.shape)
print("Negative cases: ", negative_cases.shape)

Positive cases:  (11, 5, 5, 5)
Negative cases:  (38, 5, 5, 5)


In [147]:
# Test if there are any duplicates accross the two sets
for i in range(len(positive_cases)):
    for j in range(len(negative_cases)):
        if np.array_equal(positive_cases[i], negative_cases[j]):
            print("Duplicate found!")
            print(positive_cases[i])
            print(negative_cases[j])
            break

In [148]:
# Find the number of any duplicates within the positive cases
duplicate_count = 0
for i in range(len(positive_cases)):
    for j in range(i + 1, len(positive_cases)):
        if np.array_equal(positive_cases[i], positive_cases[j]):
            duplicate_count += 1
            break

print("Duplicate count in positive cases: ", duplicate_count)

# Find the number of any duplicates within the negative cases
duplicate_count = 0
for i in range(len(negative_cases)):
    for j in range(i + 1, len(negative_cases)):
        if np.array_equal(negative_cases[i], negative_cases[j]):
            duplicate_count += 1
            break

print("Duplicate count in negative: ", duplicate_count)

Duplicate count in positive cases:  0
Duplicate count in negative:  0


In [149]:
# Print all the positive cases
for i in range(len(positive_cases)):
    print(f"Positive case {i}:")
    print(positive_cases[i])
    print()

Positive case 0:
[[[0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0.]]

 [[0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0.]]

 [[0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0.]]

 [[0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0.]]

 [[0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0.]]]

Positive case 1:
[[[0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0.]]

 [[0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0.]]

 [[0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0.]
  [0. 0. 1. 0. 0.]
  [0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0.]]

 [[0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0.]]

 [[1. 1. 1. 1. 1.]
  [1. 1. 1. 1. 1.]
  [1. 1. 1. 1. 1.]
  [1. 1. 1. 1. 1.]
  [1. 1. 1. 1. 1

In [150]:
# Print all the negative cases
for i in range(len(negative_cases)):
    print(f"Negative case {i}:")
    print(negative_cases[i])
    print()

Negative case 0:
[[[0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0.]]

 [[0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0.]]

 [[0. 0. 0. 0. 0.]
  [0. 0. 1. 0. 0.]
  [0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0.]]

 [[0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0.]]

 [[1. 1. 1. 1. 1.]
  [1. 1. 1. 1. 1.]
  [1. 1. 1. 1. 1.]
  [1. 1. 1. 1. 1.]
  [1. 1. 1. 1. 1.]]]

Negative case 1:
[[[0. 0. 0. 0. 0.]
  [0. 0. 1. 0. 0.]
  [0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0.]]

 [[0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0.]]

 [[0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0.]
  [0. 0. 1. 0. 0.]
  [0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0.]]

 [[0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0.]]

 [[0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0

In [75]:
BINARY = True

# First test if the concept can be regressed form the inputs
name = "input"
all_cases = np.concatenate([positive_cases, negative_cases])
all_labels = [1] * positive_cases.shape[0] + [0] * negative_cases.shape[0]
all_labels = np.array(all_labels)
shuffled_indices = np.arange(all_labels.shape[0])

np.random.shuffle(shuffled_indices)

all_cases = all_cases[shuffled_indices]
all_labels = all_labels[shuffled_indices]

points = all_cases.reshape(all_cases.shape[0], -1)

POSITIONS_TO_CONSIDER = points.shape[0]

print("Positions to consider: ", POSITIONS_TO_CONSIDER)

# Use the regression
score = linear_regression.perform_regression(
    points=points[:POSITIONS_TO_CONSIDER], 
    targets=all_labels[:POSITIONS_TO_CONSIDER], 
    validation_points=points[POSITIONS_TO_CONSIDER:], 
    validation_targets=all_labels[POSITIONS_TO_CONSIDER:], 
    is_binary=True,
    epochs=50,
    dynamic=True
)

print("Regression score: ", score)

# Remove the files if they exist
if os.path.exists("../concept_presences/dynamic/{}/{}/{}/".format(board_name, session_name, CONCEPT_NAME)):
    # Test if epoch folder exists
    if os.path.exists("../concept_presences/dynamic/{}/{}/{}/{}".format(board_name, session_name, CONCEPT_NAME, name)):
        # Remove all files in the epoch folder
        # Fist save the 
        for file in os.listdir("../concept_presences/dynamic/{}/{}/{}/{}".format(board_name, session_name, CONCEPT_NAME, name)):
            os.remove("../concept_presences/dynamic/{}/{}/{}/{}/{}".format(board_name, session_name, CONCEPT_NAME, name, file))

os.makedirs("../concept_presences", exist_ok=True)
os.makedirs("../concept_presences/dynamic", exist_ok=True)
os.makedirs("../concept_presences/dynamic/{}".format(board_name), exist_ok=True)
os.makedirs("../concept_presences/dynamic/{}/{}".format(board_name, session_name), exist_ok=True)
os.makedirs("../concept_presences/dynamic/{}/{}/{}".format(board_name, session_name, CONCEPT_NAME), exist_ok=True)
os.makedirs("../concept_presences/dynamic/{}/{}/{}/{}".format(board_name, session_name, CONCEPT_NAME, name), exist_ok=True)

random_suffix = ''.join(random.choices(string.ascii_uppercase + string.digits, k=10))

# Save concept presences in json file
with open("../concept_presences/dynamic/{}/{}/{}/{}/{}.json".format(board_name, session_name, CONCEPT_NAME, name, random_suffix), "w") as f:
    json.dump(score, f)

Positions to consider:  34
Epoch 1/50


2024-02-07 14:23:12.169333: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2024-02-07 14:23:12.231472: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] model_pruner failed: INVALID_ARGUMENT: Graph does not contain terminal node AssignAddVariableOp_2.


2/2 [==============================] - 2s 280ms/step - loss: 0.8064
Epoch 2/50
2/2 [==============================] - 0s 92ms/step - loss: 0.8060
Epoch 3/50
2/2 [==============================] - 0s 24ms/step - loss: 0.8058
Epoch 4/50
2/2 [==============================] - 0s 42ms/step - loss: 0.8055
Epoch 5/50
2/2 [==============================] - 0s 66ms/step - loss: 0.8053
Epoch 6/50
2/2 [==============================] - 0s 44ms/step - loss: 0.8050
Epoch 7/50
2/2 [==============================] - 0s 61ms/step - loss: 0.8048
Epoch 8/50
2/2 [==============================] - 0s 38ms/step - loss: 0.8045
Epoch 9/50
2/2 [==============================] - 0s 23ms/step - loss: 0.8043
Epoch 10/50
2/2 [==============================] - 0s 22ms/step - loss: 0.8040
Epoch 11/50
2/2 [==============================] - 0s 27ms/step - loss: 0.8037
Epoch 12/50
2/2 [==============================] - 0s 61ms/step - loss: 0.8034
Epoch 13/50
2/2 [==============================] - 0s 36ms/step - loss:

2024-02-07 14:23:16.369393: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


In [76]:
epochs_to_look_at = agents_to_sample

for epoch in epochs_to_look_at:
    path = full_model_path + model_name + "_" + str(epoch) + ".keras"
    model = ConvNet(board_size, path)

    # Will have a length equal to the sum of the numer of rows in the positive and negative cases arrays
    # And will contain 1s for positive cases and 0s for negative cases
    # Is used as labels/targets for the regression
    all_cases = np.concatenate([positive_cases, negative_cases])
    all_labels = [1] * positive_cases.shape[0] + [0] * negative_cases.shape[0]
    all_labels = np.array(all_labels)
    shuffled_indices = np.arange(all_labels.shape[0])

    np.random.shuffle(shuffled_indices)

    all_cases = all_cases[shuffled_indices]
    all_labels = all_labels[shuffled_indices]

    concept_presences = {}
    
    outputs = model.get_all_activation_values(all_cases)

    # Merge outputs
    merged_outputs = []
    for output_batch in outputs:
        for i, output_layer in enumerate(output_batch):
            if len(merged_outputs) <= i:
                merged_outputs.append([])
            merged_outputs[i].extend(output_layer)

    for i, layer_output in enumerate(merged_outputs):
        merged_outputs[i] = np.array(merged_outputs[i])
    
    outputs = merged_outputs

    # Perform regression
    concept_presence_per_layer = []
    for (i, output) in enumerate(outputs):
        points = output.reshape((output.shape[0], np.prod(output.shape[1:])))
        # So one has (n, k) samples where n is the number of positions, and k is the total number of activation values in layer i.
        print(f"Performing regression for layer {i}")
        score = linear_regression.perform_regression(
            points=points[:POSITIONS_TO_CONSIDER], 
            targets=all_labels[:POSITIONS_TO_CONSIDER], 
            validation_points=points[POSITIONS_TO_CONSIDER:], 
            validation_targets=all_labels[POSITIONS_TO_CONSIDER:], 
            is_binary=BINARY,
            epochs=50,
            dynamic=True
        )
        concept_presence_per_layer.append(score)

        print(f"The presence of {CONCEPT_NAME} in resblock {i} is {score}")
    
    concept_presences[CONCEPT_NAME] = concept_presence_per_layer

    # Remove the files if they exist
    if os.path.exists("../concept_presences/dynamic/{}/{}/{}/".format(board_name, session_name, CONCEPT_NAME)):
        # Test if epoch folder exists
        if os.path.exists("../concept_presences/dynamic/{}/{}/{}/{}".format(board_name, session_name, CONCEPT_NAME, epoch)):
            # Remove all files in the epoch folder
            # Fist save the 
            for file in os.listdir("../concept_presences/dynamic/{}/{}/{}/{}".format(board_name, session_name, CONCEPT_NAME, epoch)):
                os.remove("../concept_presences/dynamic/{}/{}/{}/{}/{}".format(board_name, session_name, CONCEPT_NAME, epoch, file))

    os.makedirs("../concept_presences", exist_ok=True)
    os.makedirs("../concept_presences/dynamic", exist_ok=True)
    os.makedirs("../concept_presences/dynamic/{}".format(board_name), exist_ok=True)
    os.makedirs("../concept_presences/dynamic/{}/{}".format(board_name, session_name), exist_ok=True)
    os.makedirs("../concept_presences/dynamic/{}/{}/{}".format(board_name, session_name, CONCEPT_NAME), exist_ok=True)
    os.makedirs("../concept_presences/dynamic/{}/{}/{}/{}".format(board_name, session_name, CONCEPT_NAME, epoch), exist_ok=True)

    random_suffix = ''.join(random.choices(string.ascii_uppercase + string.digits, k=10))

    # Save concept presences in json file
    with open("../concept_presences/dynamic/{}/{}/{}/{}/{}.json".format(board_name, session_name, CONCEPT_NAME, epoch, random_suffix), "w") as f:
        json.dump(concept_presences[CONCEPT_NAME], f)

Getting activation outputs: 100%|██████████| 2/2 [00:00<00:00, 24.98it/s]

Performing regression for layer 0
Epoch 1/50



2024-02-07 14:23:36.166084: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2024-02-07 14:23:36.208824: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] model_pruner failed: INVALID_ARGUMENT: Graph does not contain terminal node AssignAddVariableOp_2.


2/2 [==============================] - 2s 404ms/step - loss: 1.1684
Epoch 2/50
2/2 [==============================] - 0s 66ms/step - loss: 1.1650
Epoch 3/50
2/2 [==============================] - 0s 71ms/step - loss: 1.1620
Epoch 4/50
2/2 [==============================] - 0s 73ms/step - loss: 1.1592
Epoch 5/50
2/2 [==============================] - 0s 28ms/step - loss: 1.1568
Epoch 6/50
2/2 [==============================] - 0s 75ms/step - loss: 1.1547
Epoch 7/50
2/2 [==============================] - 0s 23ms/step - loss: 1.1524
Epoch 8/50
2/2 [==============================] - 0s 22ms/step - loss: 1.1497
Epoch 9/50
2/2 [==============================] - 0s 21ms/step - loss: 1.1470
Epoch 10/50
2/2 [==============================] - 0s 21ms/step - loss: 1.1442
Epoch 11/50
2/2 [==============================] - 0s 21ms/step - loss: 1.1413
Epoch 12/50
2/2 [==============================] - 0s 27ms/step - loss: 1.1385
Epoch 13/50
2/2 [==============================] - 0s 23ms/step - loss:

2024-02-07 14:23:40.329078: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2024-02-07 14:23:40.770738: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2024-02-07 14:23:40.813637: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] model_pruner failed: INVALID_ARGUMENT: Graph does not contain terminal node AssignAddVariableOp_2.


...

2024-02-07 14:25:48.395992: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2024-02-07 14:25:48.882227: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2024-02-07 14:25:48.924011: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] model_pruner failed: INVALID_ARGUMENT: Graph does not contain terminal node AssignAddVariableOp_2.


2/2 [==============================] - 2s 375ms/step - loss: 1.1446
Epoch 2/50
2/2 [==============================] - 0s 197ms/step - loss: 1.1266
Epoch 3/50
2/2 [==============================] - 0s 137ms/step - loss: 1.1132
Epoch 4/50
2/2 [==============================] - 0s 78ms/step - loss: 1.1009
Epoch 5/50
2/2 [==============================] - 0s 81ms/step - loss: 1.0909
Epoch 6/50
2/2 [==============================] - 0s 32ms/step - loss: 1.0816
Epoch 7/50
2/2 [==============================] - 0s 28ms/step - loss: 1.0730
Epoch 8/50
2/2 [==============================] - 0s 44ms/step - loss: 1.0647
Epoch 9/50
2/2 [==============================] - 0s 29ms/step - loss: 1.0563
Epoch 10/50
2/2 [==============================] - 0s 26ms/step - loss: 1.0490
Epoch 11/50
2/2 [==============================] - 0s 24ms/step - loss: 1.0416
Epoch 12/50
2/2 [==============================] - 0s 24ms/step - loss: 1.0346
Epoch 13/50
2/2 [==============================] - 0s 35ms/step - los

2024-02-07 14:25:53.196708: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2024-02-07 14:25:53.580848: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2024-02-07 14:25:53.613968: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] model_pruner failed: INVALID_ARGUMENT: Graph does not contain terminal node AssignAddVariableOp_2.


2/2 [==============================] - 1s 369ms/step - loss: 1.6406
Epoch 2/50
2/2 [==============================] - 0s 67ms/step - loss: 1.6186
Epoch 3/50
2/2 [==============================] - 0s 63ms/step - loss: 1.5999
Epoch 4/50
2/2 [==============================] - 0s 18ms/step - loss: 1.5830
Epoch 5/50
2/2 [==============================] - 0s 29ms/step - loss: 1.5673
Epoch 6/50
2/2 [==============================] - 0s 27ms/step - loss: 1.5506
Epoch 7/50
2/2 [==============================] - 0s 27ms/step - loss: 1.5371
Epoch 8/50
2/2 [==============================] - 0s 39ms/step - loss: 1.5230
Epoch 9/50
2/2 [==============================] - 0s 22ms/step - loss: 1.5104
Epoch 10/50
2/2 [==============================] - 0s 67ms/step - loss: 1.4979
Epoch 11/50
2/2 [==============================] - 0s 69ms/step - loss: 1.4860
Epoch 12/50
2/2 [==============================] - 0s 34ms/step - loss: 1.4746
Epoch 13/50
2/2 [==============================] - 0s 23ms/step - loss:

2024-02-07 14:25:57.666892: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
